# Homework 5

The designer of assignments prefer 'graphlab', but the mainstream of industry is sklearn. However, they have different outputs for the same model.

In [1]:
import pandas as pd
import numpy as np
from math import log, sqrt
from sklearn import linear_model

In [48]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [49]:
sales = pd.read_csv('./kc_house_data.csv', dtype = dtype_dict) 
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [4]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

In [16]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [19]:
model_all.coef_

array([     0.        ,      0.        ,      0.        ,    134.43931396,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,  24750.00458561,      0.        ,
        61749.10309071,      0.        ,      0.        ,     -0.        ,
            0.        ])

In [21]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [22]:
l1_penalty = np.logspace(1, 7, num=13)

In [33]:
def get_RSS(prediction, output):
    residual = output - prediction
    # square the residuals and add them up
    RS = residual*residual
    RSS = RS.sum()
    return(RSS)

for elm in l1_penalty:
    print 'Test this penalty:'
    RSS = 0
    print elm
    model = linear_model.Lasso(alpha=elm, normalize=True)
    model.fit(training[all_features], training['price']) # learning weights
    
    predictions=model.predict(validation[all_features])
    RSS = get_RSS(predictions, validation['price'])
    print RSS
    print ''

Test this penalty:
10.0
3.982133273e+14

Test this penalty:
31.6227766017
3.99041900253e+14

Test this penalty:
100.0
4.29791604073e+14

Test this penalty:
316.227766017
4.63739831045e+14

Test this penalty:
1000.0
6.45898733634e+14

Test this penalty:
3162.27766017
1.22250685943e+15

Test this penalty:
10000.0
1.22250685943e+15

Test this penalty:
31622.7766017
1.22250685943e+15

Test this penalty:
100000.0
1.22250685943e+15

Test this penalty:
316227.766017
1.22250685943e+15

Test this penalty:
1000000.0
1.22250685943e+15

Test this penalty:
3162277.66017
1.22250685943e+15

Test this penalty:
10000000.0
1.22250685943e+15



Select 10.0 as the best l1_penalty.

In [41]:
model = linear_model.Lasso(alpha=10.0, normalize=True)
model.fit(testing[all_features], testing['price']) # learning weights
    
predictions=model.predict(testing[all_features])
RSS = get_RSS(predictions, testing['price'])
print RSS

9.7244581389e+13


In [42]:
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

17

In [43]:
max_nonzeros = 7

In [45]:
l1_penalty = np.logspace(1, 4, num=20)

In [46]:
for elm in l1_penalty:
    print 'Test this penalty:'
    num_nz = 0
    print elm
    model = linear_model.Lasso(alpha=elm, normalize=True)
    model.fit(training[all_features], training['price']) # learning weights
    
    num_nz = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    print num_nz
    print ''

Test this penalty:
10.0
15

Test this penalty:
14.3844988829
15

Test this penalty:
20.6913808111
15

Test this penalty:
29.7635144163
15

Test this penalty:
42.8133239872
13

Test this penalty:
61.5848211066
12

Test this penalty:
88.586679041
11

Test this penalty:
127.42749857
10

Test this penalty:
183.298071083
7

Test this penalty:
263.665089873
6

Test this penalty:
379.269019073
6

Test this penalty:
545.559478117
6

Test this penalty:
784.759970351
5

Test this penalty:
1128.83789168
3

Test this penalty:
1623.77673919
3

Test this penalty:
2335.72146909
2

Test this penalty:
3359.81828628
1

Test this penalty:
4832.93023857
1

Test this penalty:
6951.92796178
1

Test this penalty:
10000.0
1



In [48]:
l1_penalty_min = 127.42749857
10
l1_penalty_max = 183.298071083
7

7

In [50]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [57]:
for elm in l1_penalty_values:
    RSS = 0
    predictions = 0
    num_nz = 0
    model = linear_model.Lasso(alpha=elm, normalize=True)
    model.fit(training[all_features], training['price']) # learning weights

    predictions=model.predict(validation[all_features])
    RSS = get_RSS(predictions,validation['price'])
    
    num_nz = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    if num_nz <= 7:
        print elm
        print RSS
        print num_nz
        print ''

153.892506602
4.39820045609e+14
7

156.833063051
4.40111359799e+14
7

159.773619499
4.40410061321e+14
7

162.714175947
4.40717413936e+14
7

165.654732395
4.41033335418e+14
7

168.595288843
4.41357732859e+14
7

171.535845291
4.41690625344e+14
7

174.476401739
4.42032017278e+14
7

177.416958187
4.42381906049e+14
7

180.357514635
4.42740296603e+14
7

183.298071083
4.43107216261e+14
7



In [63]:
model_lowest = linear_model.Lasso(alpha = 153.892506602, normalize=True)
model_lowest.fit(training[all_features], training['price']) # learning weights
print all_features
print model_lowest.coef_

['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
[ -0.00000000e+00  -0.00000000e+00   1.10742535e+04   1.63235168e+02
   0.00000000e+00  -0.00000000e+00  -0.00000000e+00   0.00000000e+00
   0.00000000e+00   5.08217547e+05   4.19979732e+04   0.00000000e+00
   1.16484911e+05   0.00000000e+00   0.00000000e+00  -2.62835166e+03
   0.00000000e+00]


# Homwwork 5.2

In [88]:
sales = pd.read_csv('./kc_house_data.csv', dtype = dtype_dict) 
testing = pd.read_csv('./kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('./kc_house_train_data.csv', dtype=dtype_dict)

In [66]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    # features_sframe = data_sframe[features]
    
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = data_sframe[features].as_matrix()
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_array = data_sframe[output].as_matrix()

    # this will convert the SArray into a numpy array:
    # output_array = output_sarray.to_numpy(output) # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

In [67]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [68]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


In [69]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


In [70]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


In [89]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    normalized_features = features / norms
    return (normalized_features, norms)

In [90]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [91]:
initial_weights = np.array([1., 4., 1.])

In [92]:
prediction = predict_output(simple_feature_matrix, initial_weights)

In [95]:
ro = [0 for i in range((simple_feature_matrix.shape)[1])]
for i in range((simple_feature_matrix.shape)[1]):    
    ro[i] = (simple_feature_matrix[:,i] * (output - prediction + (initial_weights[i] * simple_feature_matrix[:,i]))).sum()
print ro

[79400300.014522895, 87939470.823251754, 80966698.66623947]


In [96]:
diff = abs((ro[1]*2) - (ro[2]*2))
print('λ = (%e, %e)' %((ro[2]-diff/2+1)*2, (ro[2]+diff/2-1)*2))

λ = (1.479879e+08, 1.758789e+08)


In [97]:
print ro[1]*2
print ro[2]*2

175878941.647
161933397.332


In [98]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:,i]*(output - prediction + weights[i]*feature_matrix[:,i]))
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + (l1_penalty/2)    
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - (l1_penalty/2)    
    else:
        new_weight_i = 0.
    return new_weight_i

In [99]:
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


In [100]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights.copy()
    # converged condition variable    
    converged  = False        
    while not converged:         
        max_change = 0
        for i in range(len(weights)):
            old_weights_i = weights[i] 
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)                        
            change_i = np.abs(old_weights_i - weights[i])             
            if change_i > max_change:                
                max_change = change_i        
        if max_change < tolerance:              
            converged = True     
    return weights

In [106]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [110]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [108]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print weights

[ 21624997.95951909  63157247.20788956         0.        ]


In [111]:
predictions = predict_output(normalized_simple_feature_matrix, weights)
rss = 0
for i in range(0, len(predictions)):
    error = predictions[i] - sales['price'][i]
    rss += error * error
print rss

1.63049247672e+15


In [112]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [113]:
(all_feature_matrix, output) = get_numpy_data(training, all_features, my_output)
(normalized_all_feature_matrix, simple_norms) = normalize_features(all_feature_matrix) # normalize features
my_output = 'price'
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e7
tolerance = 1.0

In [114]:
weights_1 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty=1e7, tolerance=1)
print weights_1

[ 24429600.23440312         0.                 0.          48389174.77154896
         0.                 0.           3317511.21492165
   7329961.81171425         0.                 0.                 0.
         0.                 0.                 0.        ]


In [115]:
weights_2 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty=1e8, tolerance=1)
print weights_2

[ 71114625.71488702         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]


In [116]:
weights_3 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty=1e4, tolerance=5e5)
print weights_3

[ 78564738.34156762 -22097398.92430532  12791071.87278517
  93808088.09281193  -2013172.75704954  -4219184.93265014
   6482842.81753506   7127408.53480689   5001664.8546964   14327518.43714051
 -15770959.15237397  -5159591.22213147 -84495341.7684364    2824439.49703683]


In [117]:
(normalized_simple_feature_matrix, simple_norms) = normalize_features(all_feature_matrix) # normalize features

In [118]:
normalized_weights_1 = weights_1 / simple_norms
print normalized_weights_1[3]
normalized_weights_2 = weights_2 / simple_norms
normalized_weights_3 = weights_3 / simple_norms

161.317457646


In [120]:
(test_feature_matrix, test_output) = get_numpy_data(testing, all_features, 'price')

In [122]:
prediction =  predict_output(test_feature_matrix, normalized_weights_1)
rss = 0
for i in range(0, len(prediction)):
    error = prediction[i] - testing['price'][i]
    rss += error * error
print rss

2.7596207592e+14


In [123]:
prediction =  predict_output(test_feature_matrix, normalized_weights_2)
rss = 0
for i in range(0, len(prediction)):
    error = prediction[i] - testing['price'][i]
    rss += error * error
print rss

5.37166151497e+14


In [124]:
prediction =  predict_output(test_feature_matrix, normalized_weights_3)
rss = 0
for i in range(0, len(prediction)):
    error = prediction[i] - testing['price'][i]
    rss += error * error
print rss

2.28459958971e+14
